# Quantum Convolutional Neural Network

This is an implementation of 8 Qubit QCNN circuit testing various ansatze.

In [1]:
import pennylane as qml
from pennylane import numpy as np
np.random.seed(0)

# QCNN Circuit

Here are various unitary ansatze to test

In [2]:
# Unitraies for Convolutional Layers 
def U_TTN(theta1, theta2, wires): # 2 params
    qml.RY(theta1, wires = wires[0])
    qml.RY(theta2, wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

def U_5(params, wires): # 10 params
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])
    qml.RZ(params[2], wires = wires[0])
    qml.RZ(params[3], wires = wires[1])
    qml.CRZ(params[4], wires = [wires[1], wires[0]])
    qml.CRZ(params[5], wires = [wires[0], wires[1]])
    qml.RX(params[6], wires = wires[0])
    qml.RX(params[7], wires = wires[1])
    qml.RZ(params[8], wires = wires[0])
    qml.RZ(params[9], wires = wires[1])
    
def U_6(params, wires): # 10 params
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])
    qml.RZ(params[2], wires = wires[0])
    qml.RZ(params[3], wires = wires[1])
    qml.CRX(params[4], wires = [wires[1], wires[0]])
    qml.CRX(params[5], wires = [wires[0], wires[1]])
    qml.RX(params[6], wires = wires[0])
    qml.RX(params[7], wires = wires[1])
    qml.RZ(params[8], wires = wires[0])
    qml.RZ(params[9], wires = wires[1])

def U_9(params, wires): # 2 params
    qml.Hadamard(wires = wires[0])
    qml.Hadamard(wires = wires[1])
    qml.CZ(wires = [wires[0],wires[1]])
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])

def U_13(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CRZ(params[2], wires = [wires[1], wires[0]])
    qml.RY(params[3], wires = wires[0])
    qml.RY(params[4], wires = wires[1])
    qml.CRZ(params[5], wires = [wires[0], wires[1]])

def U_14(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CRX(params[2], wires = [wires[1], wires[0]])
    qml.RY(params[3], wires = wires[0])
    qml.RY(params[4], wires = wires[1])
    qml.CRX(params[5], wires = [wires[0], wires[1]])

def U_15(params, wires): # 4 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[1], wires[0]])
    qml.RY(params[2], wires = wires[0])
    qml.RY(params[3], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

# Unitraies for Pooling and Fully Connected Layers
def V_0(theta, wires):
    qml.CRZ(theta, wires = [wires[0], wires[1]])

def V_1(theta, wires):
    qml.CRX(theta, wires = [wires[0], wires[1]])

def F(theta, wires):
    qml.CRZ(theta, wires = [wires[0], wires[1]])

Define general layers that will be used in the circuits.

In [3]:
# Convolution Layer1
def conv_layer1(U, params):
    U(params, wires = [0,7])
    for i in range (0,8,2):
        U(params, wires = [i, i+1])
    for i in range (1,7,2):
        U(params, wires = [i, i+1])
    
def conv_layer2(U, params):
    U(params, wires = [0,2])
    U(params, wires = [4,6])
    U(params, wires = [2,4])
    U(params, wires = [0,6])
    
def pooling_layer1(V_0, V_1, params):
    for i in range(0,8,2):
        V_0(params[0], wires = [i+1, i])
    for i in range(0,8,2):
        qml.PauliX(wires = i+1)
    for i in range(0,8,2):
        V_1(params[1], wires = [i+1, i])
        

def pooling_layer2(V_0, V_1, params): # 2params
    V_0(params[0], wires = [2,0])
    V_0(params[0], wires = [6,4])
    
    qml.PauliX(wires = 2)
    qml.PauliX(wires = 6)
    
    V_1(params[1], wires = [2,0])
    V_1(params[1], wires = [6,4])

Here we define various possible embedding methods.

In [4]:
from pennylane.templates.embeddings import AmplitudeEmbedding, AngleEmbedding

def data_embedding(X, embedding_type = 'Amplitude'):
    if embedding_type == 'Amplitude':
        AmplitudeEmbedding(X, wires = range(8), normalize = True)
    elif embedding_type == 'Angle':
        AngleEmbedding(X, wires = range(8), rotation = 'Y')

Define QCNN circuit with given Unitary Ansatz and embedding method.

In [5]:
dev = qml.device('default.qubit', wires = 8)

@qml.qnode(dev)
def QCNN(X, params, U, U_params, embedding_type = 'Amplitude'):

    param1 = params[0:U_params]
    param2 = params[U_params:U_params + 2]
    param3 = params[U_params + 2: 2*U_params + 2]
    param4 = params[2*U_params + 2: 2*U_params + 4]
    param5 = params[2*U_params + 4]
    
    # Data Embedding
    data_embedding(X, embedding_type = embedding_type)
    
    #Quantum Convolutional Neural Network
    if U == 'U_TTN':
        conv_layer1(U_TTN, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_TTN, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_5':
        conv_layer1(U_5, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_5, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_6':
        conv_layer1(U_6, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_6, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_9':
        conv_layer1(U_9, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_9, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_13':
        conv_layer1(U_13, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_13, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_14':
        conv_layer1(U_14, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_14, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_15':
        conv_layer1(U_15, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_15, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    return qml.expval(qml.PauliZ(4))

# Training Quantum Circuits

In [6]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def cost(params, X, Y, U, U_params, embedding_type):
    predictions = [QCNN(x, params, U, U_params, embedding_type) for x in X]
    return square_loss(Y, predictions)

def accuracy(predictions, labels):
    acc = 0
    for l,p in zip(labels, predictions):
        if np.abs(l - p) < 0.5:
            acc = acc + 1
    return acc / len(labels)

In [7]:
def circuit_training(X_train, Y_train, U, U_params, embedding_type):
    total_params = U_params * 2 + 2 * 2 + 1
    params = np.random.randn(total_params)
    steps = 200
    learning_rate = 0.1
    batch_size =25
    opt = qml.NesterovMomentumOptimizer(learning_rate)
    
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, U, U_params, embedding_type), params)
        if it % 10 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return params

### MNIST Data loading and processing

In [31]:
import tensorflow as tf
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation

def data_load_and_process(classes = [0,1], feature_reduction = 'resize256'):
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0 #normalize the data
    
    x_train_filter_01 = np.where((y_train == classes[0]) | (y_train == classes[1]))
    x_test_filter_01 = np.where((y_test == classes[0]) | (y_test == classes[1]))
    
    x_train_01, x_test_01 = x_train[x_train_filter_01], x_test[x_test_filter_01]
    y_train_01, y_test_01 = y_train[x_train_filter_01], y_test[x_test_filter_01]
    
    y_train_01 = [1 if y ==classes[0] else -1 for y in y_train_01]
    y_test_01 = [1 if y ==classes[1] else -1 for y in y_test_01]
    
    if feature_reduction == 'resize256':   
        x_train_01 = tf.image.resize(x_train_01[:], (256, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (256, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01) 
        return x_train_01, x_test_01, y_train_01, y_test_01
    
    elif feature_reduction == 'pca8':
        x_train_01 = tf.image.resize(x_train_01[:], (784, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (784, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01)
        
        pca = PCA(8)
        x_train_01 = pca.fit_transform(x_train_01)
        x_test_01 = pca.transform(x_test_01) 
        return x_train_01, x_test_01, y_train_01, y_test_01

    elif feature_reduction == 'autoencoder8':
        latent_dim = 8 
        class Autoencoder(Model):
            def __init__(self, latent_dim):
                super(Autoencoder, self).__init__()
                self.latent_dim = latent_dim   
                self.encoder = tf.keras.Sequential([
                layers.Flatten(),
                  layers.Dense(latent_dim, activation='relu'),
                ])
                self.decoder = tf.keras.Sequential([
                layers.Dense(784, activation='sigmoid'),
                layers.Reshape((28, 28))
                ])
            def call(self, x):
                encoded = self.encoder(x)
                decoded = self.decoder(encoded)
                return decoded

        autoencoder = Autoencoder(latent_dim)
        
        autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
        autoencoder.fit(x_train, x_train,
                epochs=10,
                shuffle=True,
                validation_data=(x_test, x_test))
        return x_train_01, x_test_01, y_train_01, y_test_01


In [33]:
x_train, x_test, y_train, y_test = data_load_and_process(classes = [0,1], feature_reduction = 'pca8')

[1, -1, 1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, -1, 1, 1, 1, 1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, -1, -1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, -1, -1]


Use PCA and Autoencoder to reduce it into 8 features.

In [ ]:
pca = PCA(8)
x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)
print(x_train)

In [ ]:
from sklearn import decomposition

pca = decomposition.PCA(n_components = 8)
pca.fit(x_train)
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

# Results

In [ ]:
U = 'U_15'
U_params = 4
Embedding = 'Amplitude'
#trained_params_amp = circuit_training(x_train, y_train, U, U_params, Embedding)
#predictions = [QCNN(x, trained_params_amp) for x in x_test]
#accuracy(predictions, y_test)

In [ ]:
U = 'U_5'
U_params = 10
Embedding = 'Amplitude'
#trained_params_amp = circuit_training(x_train, y_train, U, U_params, Embedding)
#predictions = [QCNN(x, trained_params_amp) for x in x_test]
#accuracy(predictions, y_test)

In [ ]:
Unitaries = ['U_TTN, U_5, U_6, U_9, U_13, U_14, U_15']
U_num_params = [2, 10, 10, 4, 6, 6, 4]
Emedding = ['Amplitude', 'Angle', ]